# 1. Generation of the dataframe containing global measure values (bushiness and convergence measures)

In [2]:
library (BoolNet)

model_name <- "buylla"
ftype <- 'EF'

bio_fps <- read.table(sprintf("../data/data_10_Boolean_models/%s/attractors_%s.tsv", model_name, model_name), header=TRUE, sep= "\t")
rev_biofps <- bio_fps[order(nrow(bio_fps):1),]
N <- nrow(bio_fps) #Number of nodes in the network
attr_as_ints <- c()


# Function to convert a vector of bits to integer
bitsToInt <-function(x) {
  packBits(rev(c(rep(FALSE, 32-length(x)%%32), as.logical(x))), "integer")
}

# Get the indices of the biological attractors  
for (col in colnames(rev_biofps)[-1])
{
  l = as.vector(rev_biofps[[col]])
  attr_as_ints <- append(attr_as_ints, bitsToInt(l))
}

start_model <- 0
end_model <- 19

model_no <- c()
avg_indeg <- c()
g_density <- c()
avg_transient_length <- c()


for (model in start_model:end_model)
  {
  model_no <- c(model_no, model)
  bool_model <- loadNetwork(sprintf("../data/data_10_Boolean_models/%s/%s_bnet_models/model_%s.bnet", model_name, ftype, model))
  attractor_info <- getAttractors(bool_model)
  leaf_states <- setdiff(0:(2^N -1), attractor_info$stateInfo$table)
  leaf_count <- length(leaf_states)

  # Average indegree for all attractors for non leaf states
  avg_indeg <- c(avg_indeg, 2^N/(2^N-leaf_count))
  
  # g-density for all attractors
  g_density <- c(g_density, leaf_count/2^N)
  
  # transient length for all attractors
  leaf_states_index <- leaf_states + 1
  transient_lengths <- attractor_info$stateInfo$stepsToAttractor[leaf_states_index]
  avg_transient_length <- c(avg_transient_length, sum(transient_lengths)/length (transient_lengths))
    
  }
df <- data.frame(model_no, g_density, avg_indeg, avg_transient_length)
print(df)
write.table(df, file=sprintf("../data/data_10_Boolean_models/%s/results/basin_props_data/EF/%s_basin_props_example.tsv", model_name, ftype), sep = "\t", row.names=F)

   model_no g_density avg_indeg avg_transient_length
1         0 0.7265625  3.657143             3.798387
2         1 0.7265625  3.657143             3.658602
3         2 0.7500000  4.000000             4.083333
4         3 0.7968750  4.923077             3.561275
5         4 0.7578125  4.129032             3.451031
6         5 0.7421875  3.878788             3.681579
7         6 0.7421875  3.878788             3.768421
8         7 0.7812500  4.571429             3.472500
9         8 0.8125000  5.333333             3.608173
10        9 0.7109375  3.459459             3.714286
11       10 0.8281250  5.818182             3.452830
12       11 0.7812500  4.571429             3.587500
13       12 0.7812500  4.571429             3.465000
14       13 0.7500000  4.000000             3.552083
15       14 0.7031250  3.368421             3.938889
16       15 0.7343750  3.764706             4.119681
17       16 0.7812500  4.571429             4.122500
18       17 0.7890625  4.740741             3.